In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

file_path_fish = 'PManaFishTiahura.xlsx'
file_path_cor = 'TIahura_Corailgenres PMana 20230608.xlsx'

data_fish = pd.read_excel(file_path_fish)
data_cor = pd.read_excel(file_path_cor)

c:\miniconda\envs\Python_update\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [7]:
# for now delete lines where recouvrement is negative
data_cor = data_cor[data_cor['recouvrement %'] >= 0]

In [8]:
data_cor.drop(columns=['date', 'île', 'site', 'identificateur', 'observation 1', 'observation 2', 'observation 3'], inplace=True)
data_cor.rename(columns={'île x site': 'Site', 'date' : 'Date', 'genre' : 'Species'}, inplace=True)

In [9]:
# group the lines 
data_cor = data_cor.groupby(['année', 'Site', 'n° quadrat', 'Species']).sum().reset_index()
# renaame the columns
data_cor.rename(columns={'année' : 'Year'}, inplace=True)

In [11]:
data_fish['Abundance'] = data_fish['Abundance'] * data_fish['Size']

In [12]:
data_fish.drop(columns=['Date', 'Observer', 'Size'], inplace=True)

In [13]:
for specie in data_fish['Species'].unique():
    if data_fish[data_fish['Species'] == specie]['Abundance'].sum() == 0:
        data_fish.drop(data_fish[data_fish['Species'] == specie].index, inplace=True)

In [14]:
dict_fish = pd.read_csv('../trophic_groups.csv', sep=',', header = None)

In [15]:
data_fish

,Year,Site,Transect,Species,Abundance
0,2005,Moorea Tiahura 3-4,1,Abudefduf sexfasciatus,24
1,2005,Moorea Tiahura 3-4,1,Abudefduf sexfasciatus,20
2,2005,Moorea Tiahura 3-4,1,Ctenochaetus striatus,340
3,2005,Moorea Tiahura 3-4,1,Ctenochaetus striatus,190
4,2005,Moorea Tiahura 3-4,1,Forcipiger flavissimus,28
...,...,...,...,...,...
4902,2023,Moorea Tiahura 3-4,3,Centropyge loriculus,8
4903,2023,Moorea Tiahura 3-4,3,Pseudocheilinus octotaenia,7
4904,2023,Moorea Tiahura 3-4,3,Plectroglyphidodon johnstonianus,3
4905,2023,Moorea Tiahura 3-4,3,Plectroglyphidodon johnstonianus,4


In [16]:
data_fish = data_fish.replace(dict_fish[0].values, dict_fish[1].values)

In [17]:
data_fish

,Year,Site,Transect,Species,Abundance
0,2005,Moorea Tiahura 3-4,1,O,24
1,2005,Moorea Tiahura 3-4,1,O,20
2,2005,Moorea Tiahura 3-4,1,H,340
3,2005,Moorea Tiahura 3-4,1,H,190
4,2005,Moorea Tiahura 3-4,1,C,28
...,...,...,...,...,...
4902,2023,Moorea Tiahura 3-4,3,O,8
4903,2023,Moorea Tiahura 3-4,3,C,7
4904,2023,Moorea Tiahura 3-4,3,Co,3
4905,2023,Moorea Tiahura 3-4,3,Co,4


In [18]:
data_fish.drop(columns=['Site'], inplace=True)

In [19]:
# group the lines with same date and habitat
data_fish = data_fish.groupby(['Year', 'Species']).mean().reset_index()

In [20]:
data_fish.drop(columns=['Transect'], inplace=True)

In [21]:
data_fish = data_fish.pivot(index=['Year'], columns='Species', values='Abundance').reset_index()
data_fish.fillna(0, inplace=True)

In [22]:
data_fish

Species,Year,C,Co,H,O,P,Pi
0,2005,27.825243,28.915254,68.019608,66.200000,65.333333,50.062500
1,2007,45.069565,28.851852,107.000000,64.198020,118.235294,70.642857
2,2009,27.190141,13.444444,76.233333,61.085106,63.294118,47.687500
3,2011,26.484127,10.000000,52.873950,39.638889,29.421053,31.538462
4,2013,27.029851,11.375000,28.713235,35.132812,28.750000,33.687500
5,2015,35.923858,14.241379,31.783784,34.406504,47.860465,24.588235
6,2017,30.830579,13.655738,22.640845,22.352113,19.729730,32.450000
7,2019,36.259259,16.914894,35.470588,26.254386,41.193548,23.117647
8,2021,74.124378,16.260870,37.096552,40.726562,58.718750,52.380952
9,2023,41.351351,14.892857,40.478992,82.609524,64.035714,26.062500


In [23]:
data_cor['recouvrement %'] = data_cor['recouvrement %'] / 100

In [24]:
# moyenne sur les quadrats pour les coraux et sur les transects pour les poissons
data_cor = data_cor.groupby(['Year', 'Site', 'Species']).mean().reset_index()

data_cor.drop(columns=['n° quadrat'], inplace=True)

In [25]:
# conversion des sites en int
data_cor['Site'] = data_cor['Site'].replace(data_cor['Site'].unique(), [1, 2, 3, 4])


In [26]:
# moyenne sur les sites
data_cor = data_cor.groupby(['Year', 'Species']).mean().reset_index()

In [27]:
data_cor.drop(columns=['Site'], inplace=True) 

In [28]:
# set year as index
data_fish.set_index('Year', inplace=True)
data_fish.index = pd.to_datetime(data_fish.index, format='%Y')

In [29]:
# add a total column
data_fish['Total'] = data_fish.sum(axis=1)

# convert the index to datetime

data_cor = data_cor.pivot(index=['Year'], columns='Species', values='recouvrement %').reset_index()

In [30]:
data_fish

Species,C,Co,H,O,P,Pi,Total
Year,,,,,,,
2005-01-01,27.825243,28.915254,68.019608,66.200000,65.333333,50.062500,306.355938
2007-01-01,45.069565,28.851852,107.000000,64.198020,118.235294,70.642857,433.997588
2009-01-01,27.190141,13.444444,76.233333,61.085106,63.294118,47.687500,288.934643
2011-01-01,26.484127,10.000000,52.873950,39.638889,29.421053,31.538462,189.956480
2013-01-01,27.029851,11.375000,28.713235,35.132812,28.750000,33.687500,164.688399
2015-01-01,35.923858,14.241379,31.783784,34.406504,47.860465,24.588235,188.804225
2017-01-01,30.830579,13.655738,22.640845,22.352113,19.729730,32.450000,141.659004
2019-01-01,36.259259,16.914894,35.470588,26.254386,41.193548,23.117647,179.210323
2021-01-01,74.124378,16.260870,37.096552,40.726562,58.718750,52.380952,279.308064


In [31]:
data_fish['Total'] = data_fish['Total']
data_fish_total = data_fish['Total']
data_fish.drop(columns=['Total'], inplace=True)

In [32]:
# normalize the data per date
data_fish = data_fish.div(data_fish.sum(axis=1), axis=0)

data_fish['Total'] = data_fish_total

In [33]:
data_cor.set_index('Year', inplace=True)
data_cor.index = pd.to_datetime(data_cor.index, format='%Y')
data_cor.fillna(0, inplace=True)

In [34]:
# add a total column
data_cor['Total'] = data_cor.sum(axis=1)

In [35]:
data_cor

Species,Acropora,Astrea curta,Astreopora,Autres corail,Cyphastrea,Fungia,Gardineroseris,Goniastrea stelligera,Herpolitha,Leptastrea,...,Millepora,Montipora,Napopora,Pavona,Pocillopora,Porites,Porites rus,Psammocora,Sandalolitha,Total
Year,,,,,,,,,,,,,,,,,,,,,
1997-01-01,0.051389,0.002778,0.000617,0.010031,0.000000,0.001698,0.000000,0.003086,0.001440,0.008333,...,0.000000,0.015278,0.006790,0.006944,0.157099,0.053241,0.003086,0.000000,0.0,0.321811
1998-01-01,0.096296,0.004938,0.000000,0.017284,0.000000,0.006173,0.000000,0.004321,0.000000,0.002469,...,0.000000,0.011111,0.003704,0.001852,0.167284,0.024691,0.000000,0.000000,0.0,0.340123
1999-01-01,0.065741,0.002932,0.001235,0.010957,0.000000,0.005556,0.000000,0.002469,0.001029,0.010648,...,0.005556,0.026235,0.004167,0.003549,0.137191,0.061265,0.001235,0.000000,0.0,0.340998
2001-01-01,0.123213,0.004500,0.000634,0.006546,0.000000,0.006205,0.000617,0.001440,0.001405,0.012784,...,0.012963,0.026559,0.009877,0.005604,0.135932,0.079524,0.003419,0.000617,0.0,0.433092
2003-01-01,0.138580,0.000926,0.001235,0.009568,0.000000,0.008025,0.000000,0.000617,0.001235,0.011420,...,0.000000,0.011111,0.007716,0.004630,0.146605,0.092593,0.004012,0.000000,0.0,0.438272
2005-01-01,0.137963,0.002160,0.000617,0.009259,0.000000,0.008642,0.000000,0.001852,0.002160,0.013272,...,0.000000,0.009259,0.009568,0.005247,0.146914,0.090432,0.003395,0.000000,0.0,0.441049
2007-01-01,0.020988,0.001852,0.000000,0.000000,0.000000,0.003704,0.000000,0.000000,0.003704,0.000617,...,0.000000,0.003704,0.002469,0.000000,0.059259,0.030247,0.004938,0.000000,0.0,0.131481
2009-01-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.002460,0.000000,0.000000,0.000000,0.008020,0.000000,0.000000,0.000000,0.0,0.010480
2011-01-01,0.000000,0.000000,0.000000,0.001235,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000617,0.000000,0.001235,0.003086,0.000617,0.000000,0.0,0.006790


In [36]:
data_cor.to_csv('Pmana_corail_1997-2021_annuel.csv')

In [37]:
data_fish.to_csv('Pmana_poisson_2005-2023_annuel.csv')